In [1]:

from keras import models
from keras import layers


Using TensorFlow backend.


In [7]:
model  = models.Sequential()
#卷积层，参数意义分别为：
#经过这一层之后，特征图的个数，一个卷积核，产生一个特征图，第一层：32，说明有32个卷积核；第二层64，说明在第一层的特征图基础上，每张特征图有两个卷积核进行特征采集
#卷积核大小
#激活函数
#输入大小（只在开始的第一层有，后面不需要）
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(800,200,3)))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [8]:
#配置模型的损失函数、优化器、指标名称
from keras import optimizers

model.compile(loss='binary_crossentropy',          #损失函数
optimizer=optimizers.RMSprop(lr=1e-4),             #优化器
metrics=['acc'])                                   #指标名称


In [9]:
#图片的训练路径和验证路径
train_dir = r'G:\test\normal_x\typeIII1.3\train'
validation_dir = r'G:\test\normal_x\typeIII1.3\val'


In [12]:
#生成训练需要的图片和标签
from keras.preprocessing.image import ImageDataGenerator
#将图片大小调整到1以内，原先图片每个像素的格式为uint8，所以要除以255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

#根据目录的名称，生成对应的标签
#train_dir有Ⅱ型和Ⅲ型的图片
#每次生成batch_size数量的图片，图片大小为target_size
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(800, 200),              #生成图片的大小
batch_size=10,                       #一次生成图片的数量
class_mode='binary')                 #图片标签的类型

validation_generator = validation_datagen.flow_from_directory(
validation_dir,
target_size=(800, 200),              #生成图片的大小
batch_size=10,                       #一次生成图片的数量
class_mode='binary')                 #图片标签的类型


Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [13]:
#开始训练
history = model.fit_generator(
train_generator,                           #通过生成器传入图片和标签
steps_per_epoch=200,                        #每轮要传入10次，即每次100张图片进行训练
epochs=50,                                 #总共训练40轮
validation_data=validation_generator,      #通过生成器传入图片和标签进行验证
validation_steps=30)                       #每轮要传入10次，即每次30张图片进行验证

Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[288512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node mul_20 (defined at D:\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_1275]

Function call stack:
keras_scratch_graph


In [ ]:
#绘制训练精度、验证精度
#绘制训练损失、验证损失
#python画图库，类似matlab的plot
import matplotlib.pyplot as plt
acc = history.history['acc']             #得到训练的指标数据
val_acc = history.history['val_acc']     #得到验证的指标数据
loss = history.history['loss']           #得到训练损失
val_loss = history.history['val_loss']   #得到验证损失
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.savefig('accuracy_1.5_50.png')
plt.legend()                              #画图例
plt.figure()                              #另一张图
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.savefig('loss_1.5_50.png')
plt.legend()
plt.show()                                #画图，最后加上

In [ ]:
# 保存每轮的精度和损失

file = open('acc_loss_1.5_50.txt','a')
file.write('训练精度：')
for i in acc :
    file.write(str(i))
    file.write(" ")
file.write("\n")
file.write('验证精度：')
for i in val_acc :
    file.write(str(i))
    file.write(" ")


file.write("\n")
file.write('训练损失：')
for i in loss :
    file.write(str(i))
    file.write(" ")

file.write("\n")
file.write('验证损失：')
for i in val_loss :
    file.write(str(i))
    file.write(" ")

file.close()

In [ ]:
import os
import cv2 as cv
import numpy as np

III_dir = r'G:\test\normal_x\typeIII\val\III'
O_dir = r'G:\test\normal_x\typeIII\val\O'

def my_image(path):
    out = []
    filenames = os.listdir(path)
    for filename in filenames:
        image = cv.imread(os.path.join(path, filename))

        image = cv.resize(image, (100, 400))
        image = image/255.0
        out.append(image)
    return np.array(out)

imgs_III = my_image(III_dir)
imgs_O = my_image(O_dir)
ret_III = model.predict_classes(imgs_III)
ret_O = model.predict_classes(imgs_O)

ret_III = ret_III.tolist()
ret_O = ret_O.tolist()
true = ret_III.count([0])
false = ret_O.count([0])
TPR = true/len(ret_III)
FPR = false/len(ret_O)
print("TPR is :{:f} ".format(TPR))
print("FPR is :{:f} ".format(FPR))

In [ ]:
model.save('typeIII_binary_normalization_100_1.5_50.h5')